In [1]:
import time
import datetime as dt
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [ ]:
tickersdf = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [ ]:
tickers =  [i.replace('.','-') for i in tickersdf.Symbol.to_list()]
# Remove offending symbols as neede
tickers.remove('WRK')

In [ ]:
df = pd.read_csv('datasets/AAPL.csv.gz', index_col='Date', parse_dates=True)
df['date'] = df.index
buy, sell = getSignals(df)

frame = df.copy()
plt.figure(figsize=(20,5))
plt.scatter(frame.loc[buy].index, frame.loc[buy]['open'], marker='^',c='g')
plt.scatter(frame.loc[sell].index, frame.loc[sell]['open'], marker='*',c='r')
plt.plot(frame['close'], alpha=0.7)

In [7]:
def enumerate_params():
    keys = ['rsi_entry', 'rsi_exit', 'sma_period', 'stop_loss_pct', 'max_lookahead']
    params =  []
    for rsi_entry in range(15,31,5):
        for rsi_exit in range(35,46,5):
            for sma_period in range(100,201,20):
                for stop_loss_pct in range(0,6,1):
                    for max_lookahead in range(4,17,2):
                        param = dict(zip(keys, [rsi_entry, rsi_exit, sma_period, stop_loss_pct, max_lookahead]))
                        params.append(param)
    return params

def enumerate_params_products():
    keys = ['rsi_entry', 'rsi_exit', 'sma_period', 'stop_loss_pct', 'max_lookahead']
    params =  []
    r_rsi_entry = list(range(15,31,5))
    r_rsi_exit = list(range(35,46,5))
    r_sma_period = list(range(100,201,20))
    r_stop_loss_pct = list(range(0,6,1))
    r_max_lookahead = list(range(4,17,2))
    list(itertools.product(r_rsi_entry,r_rsi_exit, r_sma_period, r_stop_loss_pct, r_max_lookahead))
    return params

def in_trade(idx, trades):
    for t in trades:
        if t[1] <= idx and t[3] > idx: return True
    return False

def rsi_momo_strategy(symbol, df, params):
    #print(f'rsi_momo_strategy: {symbol} {params}')
    stop_loss_pct = params['stop_loss_pct']
    df = df.copy()
    df['rsi'] = df.ta.rsi()
    df['sma'] = df.ta.sma(params['sma_period'])
    df.dropna(inplace=True)
    
    # The DF is daily, and therefor has no weekends and timedelta is not needed.
    cutoff = df.tail(params['max_lookahead']+1).index.min()
    signals = df.loc[(df.close > df.sma) & (df.rsi < params['rsi_entry']) & (df.index < cutoff)]
    #if len(signals) == 0:
    #    print(f'rsi_momo_strategy: {symbol} No signals')
    #signals = signals.loc[(signals.index > '2014-10-01') & (signals.index < '2014-10-26')]
    trades = []
    for idx, row in signals.iterrows():
        if in_trade(idx, trades): continue
        iloc = df.index.get_loc(idx)
        buy_day = df.iloc[iloc+1]
        sell_stop = None
        if params['stop_loss_pct']:
            sell_stop = buy_day.open * (1.0 - (params['stop_loss_pct']/100))
        sell_day = None
        sell_descr = 'max_days'
        for j in range(1,params['max_lookahead']):
            this_day = df.iloc[iloc + j]
            sell_day = df.iloc[iloc + j + 1]
            if this_day.rsi > params['rsi_exit']:
                sell_descr = 'xOverRSI'
                break
            if sell_stop and this_day.low < sell_stop:
                sell_descr = 'stop_loss'
                break
        pct_return = sell_day.open / buy_day.open - 1
        #print(f'i:{i:03d} iloc:{iloc:05d} buy_on:{buy_day.name} sell_on:{sell_day.name}')
        trades.append([symbol, buy_day.name, buy_day.open, sell_day.name, sell_day.open, pct_return, sell_descr])
    return pd.DataFrame(trades, columns=['symbol', 'bdate', 'bprice', 'sdate', 'sprice', 'pct_return', 'sell_descr'])

def rsi_momo_strategy_lambda(symbol, df, params):
    #print(f'rsi_momo_strategy: {symbol} {params}')
    stop_loss_pct = params['stop_loss_pct']
    df = df.copy()
    df['rsi'] = df.ta.rsi()
    df['sma'] = df.ta.sma(params['sma_period'])
    df.dropna(inplace=True)
    
    # The DF is daily, and therefor has no weekends and timedelta is not needed.
    cutoff = df.tail(params['max_lookahead']+1).index.min()
    signals = df.loc[(df.close > df.sma) & (df.rsi < params['rsi_entry']) & (df.index < cutoff)]
    #if len(signals) == 0:
    #    print(f'rsi_momo_strategy: {symbol} No signals')
    #signals = signals.loc[(signals.index > '2014-10-01') & (signals.index < '2014-10-26')]
    trades = []
    for idx, row in signals.iterrows():
        if in_trade(idx, trades): continue
        iloc = df.index.get_loc(idx)
        buy_day = df.iloc[iloc+1]
        sell_stop = None
        if params['stop_loss_pct']:
            sell_stop = buy_day.open * (1.0 - (params['stop_loss_pct']/100))
        sell_day = None
        sell_descr = 'max_days'
        for j in range(1,params['max_lookahead']):
            this_day = df.iloc[iloc + j]
            sell_day = df.iloc[iloc + j + 1]
            if this_day.rsi > params['rsi_exit']:
                sell_descr = 'xOverRSI'
                break
            if sell_stop and this_day.low < sell_stop:
                sell_descr = 'stop_loss'
                break
        pct_return = sell_day.open / buy_day.open - 1
        #print(f'i:{i:03d} iloc:{iloc:05d} buy_on:{buy_day.name} sell_on:{sell_day.name}')
        trades.append([symbol, buy_day.name, buy_day.open, sell_day.name, sell_day.open, pct_return, sell_descr])
    return pd.DataFrame(trades, columns=['symbol', 'bdate', 'bprice', 'sdate', 'sprice', 'pct_return', 'sell_descr'])


In [8]:
def myRSI(symbol):
    df['Upmove'] = df['price_change'].apply(lambda x: x if x > 0 else 0)
    df['avg_down'] = df.Downmove.ewm(span=19).mean()

def flatten(t):
    return [item for sublist in t for item in sublist]

def load_symbol(symbol):
    file = f"datasets/{symbol}.csv.gz"
    if os.path.isfile(file):
        df = pd.read_csv(file, index_col='Date', parse_dates=True)
    else:
        period = '3mo'
        df = yf.download(symbol, progress=False, start='2005-01-01')
        df.columns = df.columns.str.lower()
        df.to_csv(file,index=True)
    df['symbol'] = symbol
    df.name = symbol
    return df

def uberdf_load_all():
    start = time.time()
    frames = {}
    for filename in os.listdir('datasets'):
        symbol = filename.split(".")[0]
        df = load_symbol(symbol)
        if df.empty: continue
        frames[symbol] = df
    end = time.time()
    print(f'Elapsed: {end - start:0.2f}')
    return frames

def uberdf_one_config(udf, params):
    print(f'uberdf_one_config: {params}')
    start = time.time()
    if len(udf) == 0:
        print(f'Loading history from filesystem')
        udf = {}
        for filename in os.listdir('datasets'):
            symbol = filename.split(".")[0]
            udf[symbol] = load_symbol(symbol)
    
    frames = []
    i = 0
    for symbol in udf.keys():
        i += 1
        #if i > 2: break
        df = udf[symbol]
        #print(f'uberdf_main {i:04d}:{symbol:6s} len={len(df)}')
        df = rsi_momo_strategy(symbol, df, params)
        if i % 5 != 0: continue
        #    print(f'backtest {i:04d}:{symbol:6s} buys:{df.bprice.count()}  mean:{df.pct_return.mean():05.2f}')
        frames.append(df)

    xdf = pd.concat(frames)
    end = time.time()
    params['elapsed'] = end - start
    params['trades'] = len(xdf)
    if len(xdf) > 0:        
        params['wins'] = wins = xdf.loc[xdf.pct_return > 0].pct_return.count()
        params['pct_wins'] = params['wins'] / len(xdf)
    else:
        params['wins'] = 0
        params['pct_wins'] = 0
    params['mean'] = xdf.pct_return.mean() 
    params['std'] = xdf.pct_return.std()
    params['sum'] = xdf.pct_return.sum()
    print(f'uberdf_one_config: results={params}')
    #print(f'uberdf_one_config: mean={params["mean"]:04.2f} std={params["sum"]:04.2f} sum={params["std"]:04.2f}')
    #print(f'Summary: wins={wins} total={total} rate={wins/total*100:04.2f} pct_return={xdf.pct_return.mean()*100:05.2f}%')
    #print(xdf.pct_return.describe())
    return xdf

def uberdf_main(shard_id=None,shard_max=None):
    start = time.time()
    configs = enumerate_params()
    i = 0
    for params in configs:
        i += 1
        params['id'] = i
        if shard_id and i % shard_max != shard_id: continue
        uberdf_one_config(udf, params)
    end = time.time()
    print(f'uberdf_main: Elapsed: {end - start:0.2f}')
    return configs

udf = uberdf_load_all()
results = uberdf_main()


Elapsed: 8.60
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 100, 'stop_loss_pct': 0, 'max_lookahead': 4}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 100, 'stop_loss_pct': 0, 'max_lookahead': 6}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 100, 'stop_loss_pct': 0, 'max_lookahead': 8}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 100, 'stop_loss_pct': 0, 'max_lookahead': 10}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 100, 'stop_loss_pct': 0, 'max_lookahead': 12}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 100, 'stop_loss_pct': 0, 'max_lookahead': 14}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entr

uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 120, 'stop_loss_pct': 1, 'max_lookahead': 10}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 120, 'stop_loss_pct': 1, 'max_lookahead': 12}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 120, 'stop_loss_pct': 1, 'max_lookahead': 14}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 120, 'stop_loss_pct': 1, 'max_lookahead': 16}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 120, 'stop_loss_pct': 2, 'max_lookahead': 4}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 120, 'stop_loss_pct': 2, 'max_lookahead': 6}
uberdf_one_config: mean=0nan std=0.00 sum=

uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 140, 'stop_loss_pct': 2, 'max_lookahead': 16}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 140, 'stop_loss_pct': 3, 'max_lookahead': 4}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 140, 'stop_loss_pct': 3, 'max_lookahead': 6}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 140, 'stop_loss_pct': 3, 'max_lookahead': 8}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 140, 'stop_loss_pct': 3, 'max_lookahead': 10}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 140, 'stop_loss_pct': 3, 'max_lookahead': 12}
uberdf_one_config: mean=0nan std=0.00 sum=0

uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 160, 'stop_loss_pct': 4, 'max_lookahead': 8}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 160, 'stop_loss_pct': 4, 'max_lookahead': 10}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 160, 'stop_loss_pct': 4, 'max_lookahead': 12}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 160, 'stop_loss_pct': 4, 'max_lookahead': 14}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 160, 'stop_loss_pct': 4, 'max_lookahead': 16}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 160, 'stop_loss_pct': 5, 'max_lookahead': 4}
uberdf_one_config: mean=0nan std=0.00 sum=

uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 180, 'stop_loss_pct': 5, 'max_lookahead': 14}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 180, 'stop_loss_pct': 5, 'max_lookahead': 16}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 4}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 6}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 8}
uberdf_one_config: mean=0nan std=0.00 sum=0nan
uberdf_one_config: {'rsi_entry': 15, 'rsi_exit': 35, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 10}
uberdf_one_config: mean=0nan std=0.00 sum=0

...

uberdf_one_config: mean=0.01 std=0.36 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 100, 'stop_loss_pct': 5, 'max_lookahead': 10}
uberdf_one_config: mean=0.00 std=0.20 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 100, 'stop_loss_pct': 5, 'max_lookahead': 12}
uberdf_one_config: mean=0.00 std=0.21 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 100, 'stop_loss_pct': 5, 'max_lookahead': 14}
uberdf_one_config: mean=0.00 std=0.23 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 100, 'stop_loss_pct': 5, 'max_lookahead': 16}
uberdf_one_config: mean=0.00 std=0.24 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 120, 'stop_loss_pct': 0, 'max_lookahead': 4}
uberdf_one_config: mean=0.00 std=0.75 sum=0.03
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 120, 'stop_loss_pct': 0, 'max_lookahead': 6}
uberdf_one_config: mean=0.01 std=1.40 sum=

uberdf_one_config: mean=0.01 std=2.71 sum=0.06
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 140, 'stop_loss_pct': 0, 'max_lookahead': 16}
uberdf_one_config: mean=0.01 std=2.41 sum=0.07
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 140, 'stop_loss_pct': 1, 'max_lookahead': 4}
uberdf_one_config: mean=0.00 std=1.15 sum=0.03
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 140, 'stop_loss_pct': 1, 'max_lookahead': 6}
uberdf_one_config: mean=0.00 std=1.06 sum=0.03
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 140, 'stop_loss_pct': 1, 'max_lookahead': 8}
uberdf_one_config: mean=0.00 std=0.97 sum=0.03
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 140, 'stop_loss_pct': 1, 'max_lookahead': 10}
uberdf_one_config: mean=0.00 std=1.05 sum=0.03
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 140, 'stop_loss_pct': 1, 'max_lookahead': 12}
uberdf_one_config: mean=0.00 std=1.08 sum=0

uberdf_one_config: mean=0.01 std=2.88 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 160, 'stop_loss_pct': 2, 'max_lookahead': 8}
uberdf_one_config: mean=0.01 std=3.26 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 160, 'stop_loss_pct': 2, 'max_lookahead': 10}
uberdf_one_config: mean=0.01 std=3.41 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 160, 'stop_loss_pct': 2, 'max_lookahead': 12}
uberdf_one_config: mean=0.01 std=3.55 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 160, 'stop_loss_pct': 2, 'max_lookahead': 14}
uberdf_one_config: mean=0.01 std=3.46 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 160, 'stop_loss_pct': 2, 'max_lookahead': 16}
uberdf_one_config: mean=0.01 std=3.54 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 160, 'stop_loss_pct': 3, 'max_lookahead': 4}
uberdf_one_config: mean=0.00 std=1.83 sum=

uberdf_one_config: mean=0.01 std=3.59 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 180, 'stop_loss_pct': 3, 'max_lookahead': 14}
uberdf_one_config: mean=0.01 std=3.44 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 180, 'stop_loss_pct': 3, 'max_lookahead': 16}
uberdf_one_config: mean=0.01 std=3.50 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 180, 'stop_loss_pct': 4, 'max_lookahead': 4}
uberdf_one_config: mean=0.01 std=2.83 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 180, 'stop_loss_pct': 4, 'max_lookahead': 6}
uberdf_one_config: mean=0.01 std=3.49 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 180, 'stop_loss_pct': 4, 'max_lookahead': 8}
uberdf_one_config: mean=0.01 std=3.78 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 180, 'stop_loss_pct': 4, 'max_lookahead': 10}
uberdf_one_config: mean=0.01 std=3.91 sum=0

uberdf_one_config: mean=0.00 std=2.78 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 200, 'stop_loss_pct': 5, 'max_lookahead': 6}
uberdf_one_config: mean=0.01 std=3.85 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 200, 'stop_loss_pct': 5, 'max_lookahead': 8}
uberdf_one_config: mean=0.01 std=4.19 sum=0.04
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 200, 'stop_loss_pct': 5, 'max_lookahead': 10}
uberdf_one_config: mean=0.01 std=4.24 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 200, 'stop_loss_pct': 5, 'max_lookahead': 12}
uberdf_one_config: mean=0.01 std=4.23 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 200, 'stop_loss_pct': 5, 'max_lookahead': 14}
uberdf_one_config: mean=0.01 std=4.39 sum=0.05
uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 45, 'sma_period': 200, 'stop_loss_pct': 5, 'max_lookahead': 16}
uberdf_one_config: mean=0.01 std=4.37 sum=

In [18]:
df = pd.DataFrame(results)
df.to_csv('./rsi_momo.results.csv')
params={'rsi_entry': 30, 'rsi_exit': 40, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 16}
df = uberdf_one_config(udf, params)
df.to_csv('./rsi_momo.best-params-trades.csv')
df

uberdf_one_config: {'rsi_entry': 30, 'rsi_exit': 40, 'sma_period': 200, 'stop_loss_pct': 0, 'max_lookahead': 16}
uberdf_one_config: mean=0.01 std=6.24 sum=0.05


,symbol,bdate,bprice,sdate,sprice,pct_return,sell_descr
0,ABBV,2014-01-28,47.080002,2014-01-31,49.939999,0.060748,xOverRSI
1,ABBV,2018-03-23,98.989998,2018-04-16,92.889999,-0.061622,max_days
2,ABBV,2020-01-27,85.050003,2020-02-05,85.059998,0.000118,xOverRSI
0,ADBE,2010-01-28,33.290001,2010-02-18,32.470001,-0.024632,xOverRSI
1,ADBE,2012-06-04,29.879999,2012-06-07,32.060001,0.072959,xOverRSI
...,...,...,...,...,...,...,...
0,ZION,2012-05-21,18.290001,2012-05-24,19.129999,0.045927,xOverRSI
1,ZION,2017-03-22,40.580002,2017-03-31,42.099998,0.037457,xOverRSI
2,ZION,2017-04-17,39.509998,2017-04-21,40.950001,0.036447,xOverRSI
3,ZION,2020-01-27,45.779999,2020-02-06,48.209999,0.053080,xOverRSI
